In [1]:
#!pip install nltk

In [2]:
import random
import pickle
import json
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [3]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\talas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\talas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\talas\AppData\Roaming\nltk_data...


True

In [4]:
lemmatizer = WordNetLemmatizer()

In [5]:
intents = json.loads(open('intents.json').read())

In [6]:
words = []
classes = []
documents = []
ignore_letters = ['?','!','.',',']

In [7]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list,intent['tag']))
        if  intent['tag'] not in classes:
            classes.append(intent['tag'])
            
print(documents)            

[(['hello'], 'greetings'), (['hey'], 'greetings'), (['hi'], 'greetings'), (['good', 'day'], 'greetings'), (['greetings'], 'greetings'), (['what', "'s", 'up', '?'], 'greetings'), (['how', 'is', 'it', 'going'], 'greetings'), (['Goodbye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['cya'], 'goodbye'), (['I', 'am', 'leaving'], 'goodbye'), (['Have', 'a', 'good', 'day'], 'goodbye'), (['bye'], 'goodbye'), (['see', 'ya'], 'goodbye'), (['cao'], 'goodbye'), (['how', 'old'], 'age'), (['how', 'old', 'is', 'Edinson'], 'age'), (['What', 'is', 'your', 'age'], 'age'), (['How', 'old', 'are', 'you', '?'], 'age'), (['age', '?'], 'age'), (['what', 'is', 'your', 'name'], 'name'), (['what', 'should', 'I', 'call', 'you'], 'name'), (['What', "'s", 'your', 'name', '?'], 'name'), (['who', 'are', 'you', '?'], 'name'), (['Can', 'you', 'tell', 'me', 'your', 'name', '?'], 'name'), (['I', "'d", 'like', 'to', 'buy', 'something'], 'shop'), (['what', 'are', 'your', 'products', '?'], 'shop'), (['what', 'do', 'y

In [8]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

print(words)

["'d", "'s", 'Can', 'Edinson', 'Goodbye', 'Have', 'How', 'I', 'See', 'What', 'a', 'about', 'age', 'am', 'are', 'buy', 'bye', 'call', 'can', 'cao', 'company', 'cya', 'day', 'do', 'going', 'good', 'greeting', 'griffyn', 'group', 'guy', 'hello', 'hey', 'hi', 'hour', 'how', 'in', 'investing', 'is', 'it', 'know', 'later', 'leaving', 'like', 'me', 'my', 'name', 'of', 'old', 'open', 'operation', 'own', 'phoenix', 'product', 'recommend', 'see', 'selling', 'share', 'should', 'something', 'stock', 'tell', 'to', 'up', 'what', 'when', 'who', 'xcaliber', 'ya', 'you', 'your']


In [9]:
classes = sorted(set(classes))
print(classes)

['age', 'goodbye', 'greetings', 'griffyn', 'hours', 'name', 'phoenix group', 'shop', 'stocks', 'xcaliber']


In [10]:
pickle.dump(words, open('words.pkl','wb'))
pickle.dump(classes, open('classes.pkl','wb'))

In [11]:
###############################################################################################################################

In [12]:
training = []
output_empty = [0]*len(classes)

In [13]:
for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

In [14]:
random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])

C:\Users\talas\AppData\Local\Temp\ipykernel_11208\4024136173.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [15]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),),activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(train_x),np.array(train_y),epochs=200,batch_size=5, verbose=1)
model.save('chatbotmodel.h5', hist)
print('Done')

Epoch 1/200
10/10 [==============================] - 1s 3ms/step - loss: 2.4001 - accuracy: 0.0000e+00
Epoch 2/200
10/10 [==============================] - 0s 2ms/step - loss: 2.2911 - accuracy: 0.2083
Epoch 3/200
10/10 [==============================] - 0s 2ms/step - loss: 2.1985 - accuracy: 0.2500
Epoch 4/200
10/10 [==============================] - 0s 2ms/step - loss: 2.1812 - accuracy: 0.3125
Epoch 5/200
10/10 [==============================] - 0s 2ms/step - loss: 2.1241 - accuracy: 0.2917
Epoch 6/200
10/10 [==============================] - 0s 2ms/step - loss: 2.1255 - accuracy: 0.2292
Epoch 7/200
10/10 [==============================] - 0s 2ms/step - loss: 1.9917 - accuracy: 0.2708
Epoch 8/200
10/10 [==============================] - 0s 2ms/step - loss: 1.9847 - accuracy: 0.3333
Epoch 9/200
10/10 [==============================] - 0s 2ms/step - loss: 1.8556 - accuracy: 0.5000
Epoch 10/200
10/10 [==============================] - 0s 2ms/step - loss: 1.7967 - accuracy: 0.4167
Epoch

10/10 [==============================] - 0s 2ms/step - loss: 0.1431 - accuracy: 0.9792
Epoch 83/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0792 - accuracy: 0.9792
Epoch 84/200
10/10 [==============================] - 0s 1ms/step - loss: 0.1106 - accuracy: 0.9583
Epoch 85/200
10/10 [==============================] - 0s 277us/step - loss: 0.1370 - accuracy: 0.9792
Epoch 86/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0984 - accuracy: 0.9792
Epoch 87/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0935 - accuracy: 1.0000
Epoch 88/200
10/10 [==============================] - 0s 2ms/step - loss: 0.1072 - accuracy: 0.9583
Epoch 89/200
10/10 [==============================] - 0s 2ms/step - loss: 0.2239 - accuracy: 0.8958
Epoch 90/200
10/10 [==============================] - 0s 2ms/step - loss: 0.1188 - accuracy: 0.9583
Epoch 91/200
10/10 [==============================] - 0s 2ms/step - loss: 0.1413 - accuracy: 0.9375
Epoch 92/20

10/10 [==============================] - 0s 1ms/step - loss: 0.0737 - accuracy: 0.9792
Epoch 164/200
10/10 [==============================] - 0s 3ms/step - loss: 0.0544 - accuracy: 0.9792
Epoch 165/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0539 - accuracy: 0.9583
Epoch 166/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0828 - accuracy: 0.9375
Epoch 167/200
10/10 [==============================] - 0s 934us/step - loss: 0.0924 - accuracy: 0.9583
Epoch 168/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0535 - accuracy: 0.9792
Epoch 169/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0556 - accuracy: 1.0000
Epoch 170/200
10/10 [==============================] - 0s 2ms/step - loss: 0.1339 - accuracy: 0.9375
Epoch 171/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0551 - accuracy: 1.0000
Epoch 172/200
10/10 [==============================] - 0s 2ms/step - loss: 0.1478 - accuracy: 0.9792
Ep